In [1]:
from math import sin, cos, tan, atan
import math
import pandas as pd
import numpy as np
from scipy import stats

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [9]:
freq = np.logspace(np.log10(20), np.log10(20000), 100)
slope = 0.1
data = [i * slope for i, f in enumerate(freq)] + 7 * np.random.rand(len(freq))

In [10]:
fig = go.Figure()

In [11]:
traces = []
traces.append(go.Scatter(x=freq, y=data))

c_slope, c_intercept, c_r, c_p, c_se = stats.linregress(x=np.log10(freq[50:98]), y=data[50:98])
line = [c_slope * math.log10(f) + c_intercept for f in freq[50:98]]
traces.append(go.Scatter(x=freq[50:98], y=line))

# line above
theta = atan(c_slope)
print("c_slope {} theta {}".format(c_slope, theta * 180 / math.pi))
f_min1 = np.power(10, np.log10(freq[50]) - 3 * sin(theta))
f_max1 = np.power(10, np.log10(freq[50]) + 3 * sin(theta))
l_min1 = line[0] + 3
l_max1 = line[-1] + 3
print("f={} f_min={} f={} f_max={}".format(freq[50], f_min1, freq[97], f_max1))

# line below
f_min2 = np.power(10, np.log10(freq[97]) + 3 * sin(theta))
f_max2 = np.power(10, np.log10(freq[97]) - 3 * sin(theta))
l_min2 = line[0] - 3
l_max2 = line[-1] - 3
print("f={} f_min={} f={} f_max={}".format(freq[50], f_min2, freq[97], f_max2))

traces.append(go.Scatter(x=freq[50:98], y=np.array(line) + 3))
traces.append(go.Scatter(x=freq[50:98], y=np.array(line) - 3))

# check
# traces.append(go.Scatter(x=[f_min1, f_max1], y=[l_min1, l_min2]))
# traces.append(go.Scatter(x=[f_min2, f_max2], y=[l_max1, l_max2]))

c_slope 3.03910814860443 theta 71.78652481978955
f=654.9098325755458 f_min=0.9257331283577944 f=17394.98005235567 f_max=463315.9121840971
f=654.9098325755458 f_min=12306077.33996979 f=17394.98005235567 f_max=24.588284525001548


In [12]:
for t in traces:
    fig.add_trace(t)

fig.update_xaxes(
    dict(
        title_text="Frequency (Hz)",
        type="log",
        range=[math.log10(20), math.log10(20000)],
        showline=True,
        dtick="D1",
    )
)
fig.show()

In [13]:
def dist_point_line(x, y, A, B, C):
    return abs(A * x + B * y + C) / math.sqrt(A * A + B * B)

In [22]:
hist_dist2 = [
    dist_point_line(math.log10(f), db, c_slope, -1, c_intercept)
    for f, db in zip(freq[50:98], data[50:98], strict=False)
]
hist_dist = [
    abs(db - (c_slope * math.log10(f) + c_intercept))
    for f, db in zip(freq[50:98], data[50:98], strict=False)
]
hist = np.histogram(hist_dist, bins=[0, 0.25, 0.5, 0.75, 1, 1.5, 2, 2.5, 3], density=False)
print(np.max(hist_dist))
counts, hist_bins = hist
bins = [
    "{:0.1f}-{:0.1f}".format(hist_bins[i], hist_bins[i + 1]) for i in range(0, len(hist_bins) - 1)
]
bins.append("{:0.1f}+".format(hist_bins[-1]))

3.462688116925663


In [23]:
histo = go.Bar(
    x=bins,
    y=counts,
)
go.Figure([histo]).show()

In [29]:
np.array(hist_dist) - np.log10(hist_dist2)

array([1.65020352, 3.42834366, 2.87346105, 2.30660028, 2.00048954,
       2.16730786, 1.30395984, 2.73699425, 1.41770631, 2.77719372,
       1.63595398, 1.90827549, 1.43994643, 3.24018029, 3.29287993,
       1.3273876 , 2.91532371, 1.40651722, 2.18615446, 1.52474847,
       1.82367524, 2.16488124, 3.21820219, 1.34840108, 1.30161064,
       3.2864679 , 1.97759985, 1.75303699, 2.34076158, 1.39857386,
       1.43856068, 2.84412836, 2.18527244, 1.74260259, 2.3916108 ,
       3.1702598 , 1.42089748, 1.63889714, 2.56011399, 2.73907362,
       2.99217184, 1.819882  , 2.94802239, 2.95698401, 1.91594956,
       3.05377173, 2.19708495, 1.49311921])